In [ ]:
!pip install tensorflow==2.19.0 tensorflow-io-gcs-filesystem==0.37.1 transformers==4.45.0 pillow matplotlib requests


In [ ]:
import tensorflow as tf
print("✅ TensorFlow loaded:", tf.__version__)


In [ ]:
import os, tarfile, requests

url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/ZjXM4RKxlBK9__ZjHBLl5A/aircraft-damage-dataset-v1.tar'
tar_filename = 'aircraft_damage_dataset_v1.tar'
extracted_folder = 'aircraft_damage_dataset_v1'

if not os.path.exists(extracted_folder):
    print("📥 Downloading dataset...")
    r = requests.get(url, stream=True)
    with open(tar_filename, "wb") as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)

    print("📦 Extracting files...")
    with tarfile.open(tar_filename) as tar:
        tar.extractall()
    print("✅ Dataset ready:", extracted_folder)
else:
    print("✅ Dataset already exists.")


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import os

# -------------------------------
# Directories (Update paths if needed)
# -------------------------------
train_dir = "aircraft_damage_dataset_v1/train"
valid_dir = "aircraft_damage_dataset_v1/valid"
test_dir  = "aircraft_damage_dataset_v1/test"

# Check if directories exist
for dir_path in [train_dir, valid_dir, test_dir]:
    if not os.path.exists(dir_path):
        raise FileNotFoundError(f"Directory not found: {dir_path}")

# -------------------------------
# Data Generators
# -------------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=8, class_mode='categorical'
)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir, target_size=(224, 224), batch_size=8, class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=(224, 224), batch_size=8, class_mode='categorical'
)

# -------------------------------
# Model Definition (VGG16 Transfer Learning)
# -------------------------------
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224,224,3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Custom classifier on top
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)  # 2 classes

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# -------------------------------
# Model Training
# -------------------------------
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=3  # Increase as needed
)

# -------------------------------
# Evaluation on Test Data
# -------------------------------
loss, acc = model.evaluate(test_generator)
print(f"✅ Test Accuracy: {acc*100:.2f}%")

# -------------------------------
# Plot Training History
# -------------------------------
plt.figure(figsize=(8,5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title("Training Accuracy Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# pick a sample image safely
sample_image = test_generator.filepaths[0]  # ensure filepaths exist
image = Image.open(sample_image).convert('RGB')

inputs = processor(image, return_tensors="pt").to(device)
out = model.generate(**inputs)

caption = processor.decode(out[0], skip_special_tokens=True)
print("🖼️ Image path:", sample_image)
print("🧠 Generated Caption:", caption)
